<a href="https://colab.research.google.com/github/ImpulsoGov/mensageria-mvp/blob/main/passo_2_match_com_equipe_e_upload_turn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Match com equipes da base de dados da Turn

#### Configurações iniciais do ambiente

In [2]:
import pandas as pd
import io
import numpy as np
from sklearn.model_selection import train_test_split
from google.oauth2 import service_account
from datetime import datetime
from google.cloud import bigquery
credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/credenciais_service_account.json')
project_id = 'predictive-keep-314223'
client = bigquery.Client(credentials= credentials,project=project_id)

#### Consulta e preparação dos dados

In [3]:
# carregando a tabela de usuários selecionados
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.historico_envio_mensagens`
"""

query_job = client.query(query)
df_historico_envio_mensagens = query_job.to_dataframe()

In [4]:
# carregando dados dos municipios da tabela do ibge
query = """
    SELECT *
    FROM `predictive-keep-314223.lista_de_codigos.municipios_ibge`
"""

query_job = client.query(query)
df_ibge = query_job.to_dataframe()

In [5]:
df_ibge['id_sus'] = df_ibge['id_sus'].astype(str)
df_historico_envio_mensagens['municipio_id_sus'] = df_historico_envio_mensagens['municipio_id_sus'].astype(str)

In [6]:
# merge da tabela de usuários com de municípios
df_historico_envio_mensagens = pd.merge(df_historico_envio_mensagens, df_ibge[['id_sus','nome','uf_sigla']], left_on=['municipio_id_sus'],right_on=['id_sus'] , how='left')

In [7]:
# carregando a view com dados da Turn io
query = """
    SELECT *
    FROM `predictive-keep-314223.ip_mensageria_camada_prata.contact_details_turnio`
"""

query_job = client.query(query)
df_contatos_turnio = query_job.to_dataframe()

In [8]:
df_contatos_turnio[['municipio', 'uf']] = df_contatos_turnio['details_municipio'].str.split(' - ', expand=True)

In [10]:
# equipes com preencimento dos dados
df_contatos_turnio_preenchido = df_contatos_turnio[df_contatos_turnio['details_equipe_nome'].notnull()]
df_contatos_turnio_preenchido = df_contatos_turnio_preenchido[['details_equipe_nome','municipio','uf']].drop_duplicates().reset_index(drop=True)
df_contatos_turnio_preenchido

,details_equipe_nome,municipio,uf
0,EQUIPE IGARAPE DO MEIO,Brejo de Areia,MA
1,-,Monsenhor Tabosa,CE
2,-,Porto Alegre,RS
3,-,Tabatinga,AM
4,ESFSB RAIMUNDO GOMES,Marechal Thaumaturgo,AC
...,...,...,...
74,EQUIPE - SANTA MARIA,Lago Verde,MA
75,EQUIPE - CENTRO DE SAUDE,Lago Verde,MA
76,FRANCISCO TEODORO MARTINS,Marajá do Sena,MA
77,EQUIPE - SANTA LUZIA,Lago Verde,MA


In [11]:
# merge das tabelas de dados dos usuários e estabelecimento
df = pd.merge(df_historico_envio_mensagens, df_contatos_turnio, left_on=['equipe_nome','municipio','uf_sigla'], right_on=['details_equipe_nome','municipio','uf'], how='left')

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4134 entries, 0 to 4133
Data columns (total 46 columns):
 #   Column                               Non-Null Count  Dtype              
---  ------                               --------------  -----              
 0   municipio                            4134 non-null   object             
 1   municipio_id_sus                     4134 non-null   object             
 2   equipe_ine                           4134 non-null   object             
 3   equipe_nome                          4134 non-null   object             
 4   linha_cuidado                        4134 non-null   object             
 5   nome_do_paciente                     4134 non-null   object             
 6   data_de_nascimento                   4134 non-null   datetime64[us]     
 7   celular_tratado                      4134 non-null   object             
 8   mvp_tipo_grupo                       4134 non-null   object             
 9   mvp_data_envio                

In [16]:
df_envio_turn = df[['municipio',
                    'uf_sigla',
                    'municipio_id_sus',
                    'equipe_ine',
                    'linha_cuidado',
                    'nome_do_paciente',
                    'data_de_nascimento',
                    'celular_tratado',
                    'mvp_tipo_grupo',
                    'mvp_data_envio',
                    'mvp_grupo',
                    'numero_visitas_ubs_ultimos_12_meses',
                    'data_ultimo_exame_cito',
                    'data_ultima_afericao_hipertensos',
                    'data_ultimo_exame_diabeticos',
                    'details_horarios_cronicos',
                    'details_telefone',
                    'details_estabelecimento_endereco',
                    'details_estabelecimento_telefone',
                    'details_estabelecimento_nome',
                    'details_horarios_cito',
                    'details_estabelecimento_documentos',
                    'details_estabelecimento_horario'
                    ]]

In [17]:
df_envio_turn = df_envio_turn.rename(columns={'details_horarios_cronicos':'horarios_cronicos',
                                              'details_telefone':'telefone',
                                              'details_estabelecimento_endereco':'estabelecimento_endereco',
                                              'details_estabelecimento_telefone':'estabelecimento_telefone',
                                              'details_estabelecimento_nome':'estabelecimento_nome',
                                              'details_horarios_cito':'horarios_cito',
                                              'details_estabelecimento_documentos':'estabelecimento_documentos',
                                              'details_estabelecimento_horario':'estabelecimento_horario'})

In [19]:
df_envio_turn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4134 entries, 0 to 4133
Data columns (total 23 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   municipio                            4134 non-null   object        
 1   uf_sigla                             4134 non-null   object        
 2   municipio_id_sus                     4134 non-null   object        
 3   equipe_ine                           4134 non-null   object        
 4   linha_cuidado                        4134 non-null   object        
 5   nome_do_paciente                     4134 non-null   object        
 6   data_de_nascimento                   4134 non-null   datetime64[us]
 7   celular_tratado                      4134 non-null   object        
 8   mvp_tipo_grupo                       4134 non-null   object        
 9   mvp_data_envio                       4134 non-null   datetime64[us]
 10  mvp_grupo   